In [239]:
import numpy as np
import matplotlib.pyplot as plt

In [240]:
p = 0.05
# p = 0.2
# p = 0.45

discount = 0.95
theta = 0.01

In [241]:
# States Space
S_space = []
N = 16  #num of states

for i in range(16):
    num = "{0:04b}".format(i)
    # S_space.append(np.array([int(i) for i in num]))
    S_space.append([int(i) for i in num])
S_space = np.array(S_space)

print(type(S_space[0][0]))

<class 'numpy.int64'>


In [242]:
# Value
V = np.zeros(16)

In [243]:
# Action Space
A = np.array([[0,0,0,0],
               [0,0,0,1],
               [0,0,1,0],
               [0,1,0,0],
               [1,0,0,0]])

# print(A)

In [244]:
# Policy
no_control_policy = []
for i in range(N):
    no_control_policy.append(A[0])     # Initialize: all a1=[0,0,0,0]

print(no_control_policy)

[array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0])]


In [245]:
# Connectivity Matrix
C = np.array([[0,0,-1,0],
               [1,0,-1,-1],
               [0,1,0,0],
               [-1,1,1,0]])

In [246]:
# v_bar mapping
def bar(vec):
    for i in range(len(vec)):
        if vec[i] > 0:          # map greater than 0 to 1
            vec[i] = 1
        else:                   # map smaller or equal to 0 to 0
            vec[i] = 0
    return vec

# print(bar(np.array([-1, 0, 1])))

In [247]:
# ⊕ component wise XOR
def mod2add(a, b):
    result = []
    for i, j in zip(a, b):
        if i==j:
            result.append(0)
        else:
            result.append(1)
    return np.array(result)

# print(mod2add(np.array([1,0,1,0]), np.array([1,0,0,1])))

In [248]:
# r(s, a, s')
def reward(state, action, next_state):
    return np.sum(next_state)*5 - np.sum(np.absolute(action))

# state = S_space[0]
# next_state = S_space[1]
# action = A[1]
# print(state, action, next_state)
# print(reward(state, action, next_state))

In [249]:
# test of exponent val

Si = S_space[1]
Sj = S_space[0]

action = A[2]
print(Si, Sj)

r = np.matmul(C, Si)
print(r)

r = bar(np.matmul(C, Si))
print(r)

r = mod2add(bar(np.matmul(C, Si)), action)
print(r)

r = Sj - mod2add(bar(np.matmul(C, Si)), action)
print(r)

r = np.absolute(Sj - mod2add(bar(np.matmul(C, Si)), action))
print(r)

r = np.sum(np.absolute(Sj - mod2add(bar(np.matmul(C, Si)), action)))
print(r)

[0 0 0 1] [0 0 0 0]
[ 0 -1  0  0]
[0 0 0 0]
[0 0 1 0]
[ 0  0 -1  0]
[0 0 1 0]
1


In [250]:
# Transition Matrix M(a)
matrix_list = []
def P(p, S_space, C, action):
    Matrix = np.zeros((N, N))

    for i in range(N):              # M(a)
        for j in range(N):
            exp = np.sum(np.absolute(S_space[j] - mod2add(bar(np.matmul(C, S_space[i])), action)))
            Matrix[i][j] = np.power(p, exp) * np.power(1-p, 4-exp)
    return Matrix

# action = A[0]
# Matrix = P(0, S_space, C, action)
# print(Matrix)

In [251]:
# Reward Matrix R(a)
def R(S_space, action):
    Matrix = np.zeros((N, N))

    for i in range(N):              # M(a)
        for j in range(N):
            Matrix[i][j] = reward(S_space[i], action, S_space[j])
    return Matrix


In [252]:
# Value Iteration

iteration_count = 0
delta = 999
while delta > theta:
    iteration_count = iteration_count+1
    V_old = V
    V_new_list = []

    for action in A:
        mat = P(p, S_space, C, action)
        rew = R(S_space, action)
        R_expected = np.matmul(np.multiply(mat, rew), np.ones(N))

        # Value
        V_new = R_expected + discount* np.matmul(mat, V_old)
        V_new_list.append(V_new)


    # update V
    V = np.amax(np.array(V_new_list), axis=0) # max of row

    # update delta
    delta = np.max(np.absolute(V-V_old))

# print(V)
print("Num of Iterations = ", iteration_count)

Num of Iterations =  142


In [253]:
# Optimal Policy
V_list = []
for action in A:
    mat = P(p, S_space, C, action)
    rew = R(S_space, action)
    R_expected = np.matmul(np.multiply(mat, rew), np.ones(N))

    # Value
    V_new = R_expected + discount* np.matmul(mat, V)
    V_list.append(V_new)

    # Record Matrix
    matrix_list.append(mat)


V_list = np.array(V_list)
optimal_action = np.argmax(V_list, axis=0)

optimal_policy = []
for i in range(N):
    optimal_policy.append(A[optimal_action[i]])

print(optimal_action)
# print(optimal_policy)
# print(policy[0])

[3 3 3 3 3 3 3 3 2 3 3 3 1 3 3 3]


In [254]:
#  Performance Evaluation

def AvgA(policy, matrix_list):

    A_sum = 0
    episode_num = 100

    # 100 episodes
    for j in range(episode_num):

        # 1 episode
        i_start = np.random.randint(0, 15)
        i_curr = i_start

        sum=0
        for i in range(200):
            # action
            action = policy[i_curr]
            sum = sum + np.sum(S_space[i_curr])

            # next state
            matrix = matrix_list[A.tolist().index(action.tolist())]
            p_next_state = matrix[i_curr]                       # probability vector of next state
            i_next = np.random.choice(16, 1, p=p_next_state)[0]    # choose next state

            # update
            i_curr = i_next

        Ai = sum/200
        A_sum = A_sum + Ai

    Avg = A_sum/episode_num
    return Avg

# print(len(matrix_list))
# print(matrix_list[4])

no_control = AvgA(no_control_policy, matrix_list)
optimal = AvgA(optimal_policy, matrix_list)

print(no_control)
print(optimal)

0.4855000000000001
2.832300000000001
